# transforms

> 2D Matrix transformations to manipulate a series of `(x,y)` coordinates.

In [ ]:
#| default_exp transforms

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import numpy as np


def apply_transform(coords_2d, xform):
    assert coords_2d.shape[1] == 2
    assert xform.shape[0] == 3
    assert xform.shape[1] == 3

    coords_full = np.concatenate([coords_2d, np.ones((coords_2d.shape[0], 1))], axis=1)
    assert coords_full.shape[0] == coords_2d.shape[0]
    assert coords_full.shape[1] == 3

    return xform.dot(coords_full.transpose()).transpose()

In [ ]:
#| export
def identity_xform():
    return np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]])

In [ ]:
#| export
def scale_xform(scale_x, scale_y):
    return np.array([[scale_x, 0, 0], [0, scale_y, 0], [0, 0, 1]])

In [ ]:
#| export
def translate_xform(translate_x, translate_y):
    return np.array([[1, 0, translate_x], [0, 1, translate_y], [0, 0, 1]])

In [ ]:
#| export
def rotate_xform(rotate_angle):
    if rotate_angle % 360 == 0:
        return identity_xform()
    theta = np.radians(rotate_angle)
    cos_theta = np.cos(theta)
    sin_theta = np.sin(theta)
    return np.array([[cos_theta, -sin_theta, 0], [sin_theta, cos_theta, 0], [0, 0, 1]])

In [ ]:
#| export
from dataclasses import dataclass


@dataclass
class BoundingBox:
    xmin: float
    xmax: float
    xrange: float
    ymin: float
    ymax: float
    yrange: float

    def normalization_xform(self, scale=1.0):
        max_range = self.xrange if self.xrange > self.yrange else self.yrange
        return scale_xform(scale / max_range, scale / max_range).dot(
            translate_xform(-self.xmin, -self.ymin)
        )

In [ ]:
#| export
def bb_rank2(coords):
    xmin = coords[:, 0].min()
    xmax = coords[:, 0].max()
    xrange = xmax - xmin
    # print(f"xrange, xmin, xmax = {xrange, xmin, xmax}")

    ymin = coords[:, 1].min()
    ymax = coords[:, 1].max()
    yrange = ymax - ymin
    # print(f"yrange, ymin, ymax = {yrange, ymin, ymax}")

    return BoundingBox(
        xmin=xmin, xmax=xmax, xrange=xrange, ymin=ymin, ymax=ymax, yrange=yrange
    )

In [ ]:
#| export
def bb_rank3(coords):
    xmin = coords[:, :, 0].min()
    xmax = coords[:, :, 0].max()
    xrange = xmax - xmin
    # print(f"xrange, xmin, xmax = {xrange, xmin, xmax}")

    ymin = coords[:, :, 1].min()
    ymax = coords[:, :, 1].max()
    yrange = ymax - ymin
    # print(f"yrange, ymin, ymax = {yrange, ymin, ymax}")

    return BoundingBox(
        xmin=xmin, xmax=xmax, xrange=xrange, ymin=ymin, ymax=ymax, yrange=yrange
    )

In [ ]:
#| hide
import nbdev

nbdev.nbdev_export()